<a href="https://colab.research.google.com/github/tzf101/BDA-Bangla-Text-Data-Augmentation/blob/main/utils_notebook/sr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Libraries



### Libraries

In [1]:
from google.colab import drive
drive.mount("/content/MyDrive", force_remount=True)

Mounted at /content/MyDrive


In [2]:
!pip install sentencepiece
!pip install rouge
!pip install sacrebleu
!pip install -U sentence-transformers
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 835.0 kB/s eta 0:00:00


In [3]:
import sentencepiece as spm
import pandas as pd

import sacrebleu
from rouge import Rouge
from sacrebleu import corpus_bleu
from bert_score import score
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import sentence_bleu

### Loading Models

In [4]:
sbert_model = SentenceTransformer('l3cube-pune/bengali-sentence-bert-nli')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.94k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/519 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Score calculation

In [5]:
def calculate_sbert_score(original, augmented):
    emb1 = sbert_model.encode(original)
    emb2 = sbert_model.encode(augmented)
    cosine_scores = util.pytorch_cos_sim(emb1, emb2)
    sbert_score = cosine_scores.item()
    return sbert_score

In [6]:
def calculate_scores(original, augmented):
    # BLEU Score
    reference = original
    candidate = augmented
    # bleu_score = sentence_bleu(reference, candidate)
    bleu_score = [sacrebleu.corpus_bleu([aug], [[orig]]).score for aug, orig in zip(augmented, original)]
    # BERTScore
    P, R, F1 = score([augmented], [original], lang="bn", rescale_with_baseline=True)

    # SBERT Score with Cosine Similarity
    emb1 = sbert_model.encode(original)
    emb2 = sbert_model.encode(augmented)
    cosine_scores = util.pytorch_cos_sim(emb1, emb2)
    sbert_score = cosine_scores.item()

    return bleu_score, F1.item(), sbert_score

### SR function

In [7]:
!pip install bnlp

  Preparing metadata (setup.py) ... done
  Created wheel for bnlp: filename=bnlp-0.8-py3-none-any.whl size=32803 sha256=416158f8b3aa183cd6b83804e27b8470b765c4c17d97bff77706b3c7ad5f8e33
  Stored in directory: /root/.cache/pip/wheels/64/c7/7e/272284eb8c800fbb5306bdf7904ae9f99a64812fc615f471b2
Successfully built bnlp


In [8]:
!pip install bnlp-toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.6 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=0c2d4bed7ed850ee720ca24414f34a4eb57f33610b3923c3e618795c4e9bff65
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [9]:
import random
from random import shuffle
import re

In [10]:
from bnlp import BengaliCorpus as corpus

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
from bnlp import BengaliWord2Vec

In [12]:
def get_synonyms(word):
    synonyms = set()
    bwv = BengaliWord2Vec()
    similar_words = bwv.get_most_similar_words(word, topn=10)
    for word in similar_words:
        synonyms.add(word[0])
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

In [13]:
import random

class SR():
    def __init__(self, stopwords):
        self.stopwords = stopwords

    def augment(self, text, n, debug=False):
        words = text.split()
        new_words = words.copy()
        random_word_list = list(set([word for word in words if word not in self.stopwords]))
        random.shuffle(random_word_list)
        num_replaced = 0
        for random_word in random_word_list:
            try:
                synonyms = get_synonyms(random_word)
                if len(synonyms) >= 1:
                    synonym = random.choice(list(synonyms))
                    new_words = [synonym if word == random_word else word for word in new_words]
                    num_replaced += 1
            except KeyError:
                # Ignore words not present in the vocabulary
                continue
            if num_replaced >= n:
                break

        output = ' '.join(new_words)
        if debug:
            output += "(sr)"
        return output


### Testing

In [14]:
stopwords = set(corpus.stopwords)
sr = SR(stopwords)

text = "সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে।"
augmented_text = sr.augment(text, n=2)
print(augmented_text)

Extracting: bangla_word2vec_gen4.zip:   0%|          | 0/8 [00:00<?, ?it/s]

সঠিক তদন্ত করতে হবে। বিচারের অধীন আনতে হবে যে এই কাজ টা স্টুপিড


### Apply SR on dataset

In [15]:
# Define a function to apply paraphrasing
def apply_sr(row):
    return sr.augment(row['original_sentence'], n=2)

In [16]:
def sr_and_evaluate_dataset(file_path, original_col_name, new_col_name):
    # Load the dataset
    df = pd.read_csv(file_path)
    df = df.rename(columns={original_col_name: 'original_sentence'})

    # Apply the text augmentation function
    df[new_col_name] = df.apply(apply_sr, axis=1)

    # Compute ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(df[new_col_name], df["original_sentence"])
    rouge_df = pd.DataFrame([
        {
            'r1f1': score['rouge-1']['f'],
            'r2f1': score['rouge-2']['f'],
            'rlf1': score['rouge-l']['f'],
        }
        for score in rouge_scores
    ])

    # Compute SacreBLEU scores
    sacrebleu_scores = [sacrebleu.corpus_bleu([aug], [[orig]]).score for aug, orig in zip(df[new_col_name], df["original_sentence"])]
    df["sacrebleu_score"] = sacrebleu_scores

    # SBERT Score with Cosine Similarity
    df["sbert_score"] = [calculate_sbert_score(orig, aug) for orig, aug in zip(df["original_sentence"], df[new_col_name])]

    # BERTScore
    P, R, F1 = score(df[new_col_name], df["original_sentence"], lang="en", rescale_with_baseline=True)
    df["bertscore_f1"] = F1.tolist()

    # Combine the dataframes
    result_df = pd.concat([df, rouge_df], axis=1)
    result_df["method"] = "sr2"

    return result_df

# Running on dataset

In [17]:
file_path = '/content/MyDrive/MyDrive/Research/Thesis: BDA/Main/evaluation/(old)Youtube/Datasets/yt_sentiment_train_10.csv'

In [ ]:
result_df = sr_and_evaluate_dataset(file_path, 'sentence1', 'augmented_sentence')

In [ ]:
result_df

### Saving augmented dataset

In [ ]:
result_df.to_csv('/content/MyDrive/MyDrive/Research/Thesis: BDA/Main/evaluation/(old)Youtube/Datasets/yt_sentiment_train_10_sr.csv', index=False)